# 🛡️ Phishing Risk Scoring Engine
### Phase 5 — Multi-Modal Detection (NLP + URL)
---
**How it works:**
```
Email Text ──► DistilBERT (NLP Model)  ──► NLP Score  (0.0 – 1.0)
URL        ──► Random Forest (URL Model) ──► URL Score  (0.0 – 1.0)

Final Score = (0.6 × NLP Score) + (0.4 × URL Score)

Score > 0.7  ──► 🚨 PHISHING
Score ≤ 0.7  ──► ✅ SAFE
```
---
📂 **Files needed (upload when asked):**
- `phishing_detector_model.zip` — from Phase 3
- `url_models.zip` — from Phase 4

⚠️ **Enable GPU:** `Runtime → Change runtime type → T4 GPU → Save`

---

## ✅ Step 1 — Check GPU & Install Libraries

In [ ]:
import torch
if torch.cuda.is_available():
    print('✅ GPU ready:', torch.cuda.get_device_name(0))
else:
    print('⚠️  No GPU — NLP model will run on CPU (slower but works)')

In [ ]:
!pip install transformers scikit-learn -q
print('✅ Libraries installed!')

## ✅ Step 2 — Upload Model Files

In [ ]:
from google.colab import files

# Upload Phase 3 NLP model zip
print('📂 Upload your phishing_detector_model.zip (from Phase 3)')
uploaded = files.upload()
print('✅ Uploaded:', list(uploaded.keys()))

In [ ]:
# Upload Phase 4 URL models zip
print('📂 Upload your url_models.zip (from Phase 4)')
uploaded2 = files.upload()
print('✅ Uploaded:', list(uploaded2.keys()))

In [ ]:
# Unzip both files
!unzip -q phishing_detector_model.zip -d ./
!unzip -q url_models.zip -d ./

# Verify all files are present
import os
print('📁 NLP model files:')
for f in os.listdir('./phishing_detector_model'):
    print(f'   {f}')

print('\n📁 URL model files:')
for f in ['url_rf_model.pkl', 'url_lr_model.pkl', 'url_scaler.pkl']:
    status = '✅' if os.path.exists(f) else '❌ MISSING'
    print(f'   {f} {status}')

## ✅ Step 3 — Load NLP Model (DistilBERT)

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Loading DistilBERT NLP model...')
nlp_tokenizer = DistilBertTokenizer.from_pretrained('./phishing_detector_model')
nlp_model     = DistilBertForSequenceClassification.from_pretrained('./phishing_detector_model')
nlp_model.to(device)
nlp_model.eval()
print(f'✅ NLP model loaded on: {device}')


def get_nlp_score(email_text):
    """Returns phishing probability (0.0 to 1.0) from email text."""
    inputs = nlp_tokenizer(
        email_text,
        return_tensors='pt',
        truncation=True,
        max_length=256,
        padding=True
    ).to(device)
    with torch.no_grad():
        outputs = nlp_model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1)
    return probs[0][1].item()  # phishing probability


# Quick test
test_score = get_nlp_score("Your account has been suspended. Click here to verify.")
print(f'\nTest NLP score (phishing email): {test_score:.4f}')
print('✅ NLP model working!')

## ✅ Step 4 — Load URL Model (Random Forest)

In [ ]:
import pickle
import re
import math
from urllib.parse import urlparse

# Load models
with open('url_rf_model.pkl', 'rb') as f:
    url_model = pickle.load(f)
with open('url_scaler.pkl', 'rb') as f:
    url_scaler = pickle.load(f)

print('✅ URL model loaded!')

# Feature names must match Phase 4 training order exactly
URL_FEATURE_NAMES = [
    'url_length', 'num_dots', 'num_hyphens', 'num_underscores',
    'num_slashes', 'num_question_marks', 'num_equal_signs', 'num_at_signs',
    'num_ampersands', 'num_percent', 'num_digits', 'digit_ratio',
    'num_suspicious_words', 'has_ip_address', 'is_https', 'domain_length',
    'num_subdomains', 'has_suspicious_tld', 'path_length', 'path_depth',
    'has_exe_extension', 'has_php', 'query_length', 'num_query_params',
    'has_non_standard_port', 'domain_entropy', 'tld_encoded'
]

def extract_url_features(url):
    """Extract 27 features from a URL — same as Phase 4."""
    f = {}
    f['url_length']            = len(url)
    f['num_dots']              = url.count('.')
    f['num_hyphens']           = url.count('-')
    f['num_underscores']       = url.count('_')
    f['num_slashes']           = url.count('/')
    f['num_question_marks']    = url.count('?')
    f['num_equal_signs']       = url.count('=')
    f['num_at_signs']          = url.count('@')
    f['num_ampersands']        = url.count('&')
    f['num_percent']           = url.count('%')
    f['num_digits']            = sum(c.isdigit() for c in url)
    f['digit_ratio']           = f['num_digits'] / len(url) if len(url) > 0 else 0
    suspicious_words = ['login','signin','verify','secure','account',
                        'update','banking','confirm','password','pay',
                        'free','lucky','win','bonus','click']
    f['num_suspicious_words']  = sum(w in url.lower() for w in suspicious_words)
    f['has_ip_address']        = int(bool(re.search(r'(https?://)?(\d{1,3}\.){3}\d{1,3}', url)))
    try:
        parsed = urlparse(url if url.startswith('http') else 'http://' + url)
    except:
        parsed = urlparse('')
    f['is_https']              = int(parsed.scheme == 'https')
    netloc                     = parsed.netloc or ''
    domain                     = netloc.split(':')[0]
    f['domain_length']         = len(domain)
    f['num_subdomains']        = max(0, len(domain.split('.')) - 2)
    parts                      = domain.split('.')
    tld                        = parts[-1].lower() if len(parts) >= 2 else ''
    suspicious_tlds            = ['ru','xyz','tk','ml','ga','cf','gq','top',
                                   'work','click','link','info','biz','online']
    f['has_suspicious_tld']    = int(tld in suspicious_tlds)
    path                       = parsed.path or ''
    f['path_length']           = len(path)
    f['path_depth']            = path.count('/')
    f['has_exe_extension']     = int(bool(re.search(r'\.(exe|sh|bat|cmd|msi|ps1|vbs|js)$', path.lower())))
    f['has_php']               = int('.php' in path.lower())
    query                      = parsed.query or ''
    f['query_length']          = len(query)
    f['num_query_params']      = query.count('&') + 1 if query else 0
    port                       = parsed.port
    f['has_non_standard_port'] = int(port is not None and port not in [80, 443])
    if domain:
        probs = [domain.count(c) / len(domain) for c in set(domain)]
        f['domain_entropy']    = -sum(p * math.log2(p) for p in probs if p > 0)
    else:
        f['domain_entropy']    = 0
    f['tld_encoded']           = hash(tld) % 100
    return f


def get_url_score(url):
    """Returns phishing probability (0.0 to 1.0) from a URL."""
    import pandas as pd
    features = extract_url_features(url)
    X = pd.DataFrame([features])[URL_FEATURE_NAMES]
    proba = url_model.predict_proba(X)[0]
    return proba[1]  # phishing probability


# Quick test
test_url_score = get_url_score('http://paypal-verify.tk/login.php')
print(f'Test URL score (phishing URL): {test_url_score:.4f}')
print('✅ URL model working!')

## ✅ Step 5 — Build the Risk Scoring Engine

In [ ]:
import re as _re

# Weights for combining scores
NLP_WEIGHT = 0.6
URL_WEIGHT = 0.4
THRESHOLD  = 0.7  # above this = phishing


def extract_urls_from_text(text):
    """Extract all URLs found inside email text."""
    pattern = r'https?://[^\s<>"{}|\\^`\[\]]+'  
    return _re.findall(pattern, text)


def risk_score_engine(email_text, url=None):
    """
    Multi-modal phishing detection engine.

    Args:
        email_text : raw email body text
        url        : (optional) URL to analyse separately.
                     If None, URLs are auto-extracted from email_text.

    Returns:
        dict with all scores and final verdict
    """

    # ── 1. NLP Score from email text ────────────────────────────
    nlp_score = get_nlp_score(email_text)

    # ── 2. URL Score ─────────────────────────────────────────────
    if url:
        urls_found = [url]
    else:
        urls_found = extract_urls_from_text(email_text)

    if urls_found:
        # Average score across all URLs found
        url_scores = [get_url_score(u) for u in urls_found]
        url_score  = sum(url_scores) / len(url_scores)
    else:
        # No URL found — rely only on NLP, reduce URL weight
        url_score  = 0.0

    # ── 3. Final Combined Score ──────────────────────────────────
    if urls_found:
        final_score = (NLP_WEIGHT * nlp_score) + (URL_WEIGHT * url_score)
    else:
        # If no URL, use NLP score alone with same threshold
        final_score = nlp_score

    # ── 4. Verdict ───────────────────────────────────────────────
    is_phishing = final_score > THRESHOLD

    # Risk level
    if final_score > 0.85:
        risk_level = '🔴 HIGH RISK'
    elif final_score > 0.7:
        risk_level = '🟠 MEDIUM-HIGH RISK'
    elif final_score > 0.4:
        risk_level = '🟡 MEDIUM RISK'
    else:
        risk_level = '🟢 LOW RISK'

    return {
        'nlp_score':    round(nlp_score, 4),
        'url_score':    round(url_score, 4),
        'final_score':  round(final_score, 4),
        'verdict':      '🚨 PHISHING' if is_phishing else '✅ SAFE',
        'risk_level':   risk_level,
        'urls_found':   urls_found,
        'is_phishing':  is_phishing
    }


def print_result(email_text, url=None, label=None):
    """Run engine and print a nicely formatted result."""
    result = risk_score_engine(email_text, url)
    short  = email_text[:80] + '...' if len(email_text) > 80 else email_text

    print('=' * 65)
    print(f'📧 Email:       "{short}"')
    if result['urls_found']:
        print(f'🔗 URLs found:  {result["urls_found"]}')
    else:
        print(f'🔗 URLs found:  None')
    print(f'─' * 65)
    print(f'NLP Score:    {result["nlp_score"]:.4f}  (weight: {NLP_WEIGHT})')
    print(f'URL Score:    {result["url_score"]:.4f}  (weight: {URL_WEIGHT})')
    print(f'Final Score:  {result["final_score"]:.4f}  (threshold: {THRESHOLD})')
    print(f'─' * 65)
    print(f'Risk Level:   {result["risk_level"]}')
    print(f'Verdict:      {result["verdict"]}')
    if label:
        match = '✅ CORRECT' if (label == 'phishing') == result['is_phishing'] else '❌ WRONG'
        print(f'Actual Label: {label.upper()} → {match}')
    print()


print('✅ Risk Scoring Engine ready!')
print(f'   Formula: ({NLP_WEIGHT} × NLP Score) + ({URL_WEIGHT} × URL Score)')
print(f'   Threshold: > {THRESHOLD} = Phishing')

## ✅ Step 6 — Test the Engine with Sample Emails

In [ ]:
print('🧪 RUNNING PHISHING DETECTION TESTS')
print()

# Test 1 — Phishing email with phishing URL
print_result(
    email_text=(
        "URGENT: Your PayPal account has been suspended. "
        "Please verify your identity immediately at "
        "http://paypal-secure-verify.tk/login.php "
        "or your account will be permanently closed."
    ),
    label='phishing'
)

# Test 2 — Safe email, no URL
print_result(
    email_text=(
        "Hi Sarah, just following up on the project report. "
        "Could you please send me the updated figures by Thursday? Thanks!"
    ),
    label='safe'
)

# Test 3 — Phishing email, URL provided separately
print_result(
    email_text=(
        "Your bank account requires immediate verification. "
        "Click the link below to update your details and avoid suspension."
    ),
    url='http://192.168.1.1:8080/bank/verify-account.php',
    label='phishing'
)

# Test 4 — Safe email with legitimate URL
print_result(
    email_text=(
        "Please find the latest documentation update here: "
        "https://docs.github.com/en/get-started "
        "Let me know if you have any questions."
    ),
    label='safe'
)

# Test 5 — Tricky phishing (urgent keywords + suspicious URL)
print_result(
    email_text=(
        "Congratulations! You have won a FREE iPhone 15. "
        "Claim your prize now before it expires: "
        "http://free-iphone-winner.click/claim?user=winner&prize=iphone"
    ),
    label='phishing'
)

## ✅ Step 7 — Visualise Score Breakdown

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

# Run all test cases and collect results
test_cases = [
    {
        'name': 'PayPal Phishing',
        'text': 'URGENT: Your PayPal account has been suspended. Verify at http://paypal-secure-verify.tk/login.php',
        'actual': 'Phishing'
    },
    {
        'name': 'Normal Work Email',
        'text': 'Hi Sarah, could you please send me the updated figures by Thursday? Thanks!',
        'actual': 'Safe'
    },
    {
        'name': 'Bank Verification',
        'text': 'Your account requires immediate verification. http://192.168.1.1:8080/bank/verify.php',
        'actual': 'Phishing'
    },
    {
        'name': 'GitHub Docs Link',
        'text': 'Please find the documentation here: https://docs.github.com/en/get-started',
        'actual': 'Safe'
    },
    {
        'name': 'iPhone Prize Scam',
        'text': 'Congratulations! FREE iPhone 15. Claim now: http://free-iphone-winner.click/claim',
        'actual': 'Phishing'
    },
]

results = []
for tc in test_cases:
    r = risk_score_engine(tc['text'])
    r['name']   = tc['name']
    r['actual'] = tc['actual']
    results.append(r)

# ── Bar chart of scores ──────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

names       = [r['name'] for r in results]
nlp_scores  = [r['nlp_score'] for r in results]
url_scores  = [r['url_score'] for r in results]
final_scores= [r['final_score'] for r in results]
colors      = ['tomato' if r['is_phishing'] else 'steelblue' for r in results]

x = np.arange(len(names))
w = 0.25

axes[0].bar(x - w, nlp_scores,   w, label='NLP Score',   color='mediumpurple', alpha=0.85)
axes[0].bar(x,     url_scores,   w, label='URL Score',   color='steelblue',   alpha=0.85)
axes[0].bar(x + w, final_scores, w, label='Final Score', color='tomato',      alpha=0.85)
axes[0].axhline(y=0.7, color='black', linestyle='--', linewidth=1.5, label='Threshold (0.7)')
axes[0].set_xticks(x)
axes[0].set_xticklabels(names, rotation=20, ha='right', fontsize=9)
axes[0].set_ylabel('Score')
axes[0].set_ylim(0, 1.1)
axes[0].set_title('NLP vs URL vs Final Score', fontsize=13)
axes[0].legend(fontsize=9)
axes[0].grid(axis='y', alpha=0.3)

# ── Gauge-style final score chart ───────────────────────────────
axes[1].barh(names, final_scores, color=colors, edgecolor='black', linewidth=0.5)
axes[1].axvline(x=0.7, color='black', linestyle='--', linewidth=2, label='Threshold (0.7)')
axes[1].set_xlabel('Final Risk Score')
axes[1].set_xlim(0, 1.0)
axes[1].set_title('Final Risk Score by Email', fontsize=13)
axes[1].legend(fontsize=9)
for i, (score, r) in enumerate(zip(final_scores, results)):
    axes[1].text(score + 0.01, i, f"{score:.2f} — {r['verdict']}", va='center', fontsize=9)
axes[1].grid(axis='x', alpha=0.3)

red_patch  = mpatches.Patch(color='tomato',    label='Predicted: Phishing')
blue_patch = mpatches.Patch(color='steelblue', label='Predicted: Safe')
axes[1].legend(handles=[red_patch, blue_patch], fontsize=9)

plt.tight_layout()
plt.show()

# ── Accuracy summary ────────────────────────────────────────────
correct = sum(1 for r in results
              if r['is_phishing'] == (r['actual'] == 'Phishing'))
print(f'\n✅ Test accuracy: {correct}/{len(results)} correct ({correct/len(results)*100:.0f}%)')

## ✅ Step 8 — Interactive Demo (Try Your Own Input)

In [ ]:
# ── Paste any email + URL here and run this cell ─────────────────

my_email = """Your account is at risk. Please click here to verify your details immediately."""

my_url = None  # paste a URL string here, or leave as None to auto-detect from email

# ─────────────────────────────────────────────────────────────────
print_result(my_email, url=my_url)

## ✅ Step 9 — Save the Complete Engine

In [ ]:
import pickle

# Save URL model + scaler (NLP model is already saved as a folder)
engine_config = {
    'nlp_weight':  NLP_WEIGHT,
    'url_weight':  URL_WEIGHT,
    'threshold':   THRESHOLD,
    'url_features': URL_FEATURE_NAMES
}

with open('engine_config.pkl', 'wb') as f:
    pickle.dump(engine_config, f)

print('✅ Engine config saved!')

# Zip everything together
!zip -r phase5_risk_engine.zip \
    phishing_detector_model/ \
    url_rf_model.pkl \
    url_lr_model.pkl \
    url_scaler.pkl \
    engine_config.pkl

from google.colab import files
files.download('phase5_risk_engine.zip')
print('✅ Download started — this zip contains your complete detection engine!')

---
## 🎉 Phase 5 Complete!

| Step | Task | Status |
|------|------|--------|
| 1 | GPU check + install | ✅ |
| 2 | Upload Phase 3 + Phase 4 models | ✅ |
| 3 | Load DistilBERT NLP model | ✅ |
| 4 | Load Random Forest URL model | ✅ |
| 5 | Build risk scoring engine | ✅ |
| 6 | Test with 5 sample emails | ✅ |
| 7 | Visualise score breakdown | ✅ |
| 8 | Interactive demo | ✅ |
| 9 | Save complete engine | ✅ |

**Your multi-modal detection system:**
```
Final Score = (0.6 × NLP Score) + (0.4 × URL Score)
Score > 0.7  →  🚨 PHISHING
Score ≤ 0.7  →  ✅ SAFE
```

**Files in phase5_risk_engine.zip:**
- `phishing_detector_model/` — DistilBERT NLP model
- `url_rf_model.pkl` — Random Forest URL model
- `url_scaler.pkl` — Feature scaler
- `engine_config.pkl` — Weights & threshold config
---